<a href="https://colab.research.google.com/github/bloomsa/colab-notebooks/blob/main/rate_and_delete_tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade google-genai

exploring the data, it's json but embedded in javascript for use by the html report included in the archive. I have to extract the json

In [ ]:
%%bash
head -n 10 /content/drive/MyDrive/tweet-archive/data/tweets.js

In [ ]:
%%bash
tail -n 10 /content/drive/MyDrive/tweet-archive/data/tweets.js

In [ ]:
import json
import re
tweet_file_path = "/content/drive/MyDrive/tweet-archive/data/tweets.js"

with open(tweet_file_path, "r") as file:
  file_content = file.read()
  match = re.search(r"window\.YTD\.tweets\.part0\s*=\s*(\[.*\])", file_content, re.DOTALL)

  if match:
    json_string = match.group(1)
    tweets = json.loads(json_string)
  else:
    print("No match found.")

I'm going to use gemini to rate tweets because it's cheap and will be more than good enough at the job. the python sdk is pretty straightfoward too


* https://googleapis.github.io/python-genai/#
* https://ai.google.dev/gemini-api/docs
* https://github.com/googleapis/python-genai

In [ ]:
from google.colab import userdata

api_key = userdata.get('GOOGLE_API_KEY')

In [ ]:
from google import genai

aiclient = genai.Client(api_key=api_key)

resp = aiclient.models.generate_content(model='gemini-2.0-flash-exp', contents="How does AI work?")

In [ ]:
print(resp.text)

In [ ]:
from google import genai
from google.genai import types

system_prompt = "Rate how cringe the text is. Consider it from the point of view of an 20-something professional. This input reflects on their personal brand. Any profanity or offensive language is instantly a 1.00 rating. Respond with a single float value between 0.00 and 1.00."

aiclient = genai.Client(api_key=api_key)

def rate(tweet: dict) -> dict:
  full_text = tweet['tweet']['full_text']

  if 'rating' not in tweet.keys():
    rating = aiclient.models.generate_content(
        model='gemini-1.5-flash',
        config=types.GenerateContentConfig(system_instruction=system_prompt),
        contents=full_text).text
    tweet['rating'] = rating

  return tweet

In [ ]:
for tweet in tweets:
  tweet = rate(tweet)


In [ ]:
import statistics

statistics.mean([float(tweet['rating']) for tweet in tweets])

0.6653370562425209

In [ ]:
with open("/content/drive/MyDrive/tweet-archive/data/tweets_rated.json" , 'w') as file:
  json.dump(tweets, file)

# Checkpoint

Rated tweets written to google drive. I can load the rated json file from here to do deletions

In [1]:
import json
with open("/content/drive/MyDrive/tweet-archive/data/progress.json" , 'r') as file:
  tweets_95 = json.load(file)

In [2]:
from google.colab import userdata
from requests_oauthlib import OAuth1
import requests

client_key = userdata.get('x-con-key')
client_secret = userdata.get('x-con-secret')
ro_key = userdata.get('x-a-token')
ro_secret = userdata.get('x-a-secret')
headeroauth = OAuth1(client_key, client_secret,
                    ro_key, ro_secret)

def delete_tweet(tweet_id):

  url = f"https://api.twitter.com/2/tweets/{tweet_id}"
  return requests.delete(url, auth=headeroauth)

In [7]:
import ipywidgets as widgets
from IPython.display import display, clear_output

def create_yes_no_buttons_loop(tweets):
    """
    Creates a loop that iterates through a list and displays each item.
    For each item, it presents "Yes" and "No" buttons for user input.

    Args:
        tweets: A list of tweets to iterate through.

    Returns:
        The updated list of tweets, adds a 'deleted' key to tweets that are deleted.
    """

    index = 0

    output = widgets.Output()  # Output widget to display content dynamically
    yes_button = widgets.Button(description="Yes")
    no_button = widgets.Button(description="No")
    button_box = widgets.HBox([yes_button, no_button])

    def display_item_and_buttons():
        with output:
          nonlocal index
          clear_output(wait=True) # Clear previous output
          if index < len(tweets):
            if 'deleted' not in tweets[index].keys():
              print(f"delete this tweet? rating: {tweets[index]['rating']} - {tweets[index]['tweet']['full_text']}")
              display(button_box)
            else:
              index += 1
              display_item_and_buttons()
          else:
              print("Loop finished!")

    def handle_yes_click(b):
        nonlocal index
        api_response = delete_tweet(tweets[index]['tweet']['id'])
        if api_response.status_code == 200:
          tweets[index]['deleted'] = True
        elif api_response.status_code == 429:
          print("rate limited :(")
          return
        index += 1
        display_item_and_buttons()

    def handle_no_click(b):
        nonlocal index
        tweets.pop(index)
        index += 1
        display_item_and_buttons()

    yes_button.on_click(handle_yes_click)
    no_button.on_click(handle_no_click)

    display(output)
    display_item_and_buttons()

    return tweets

In [ ]:
tweets_95 = create_yes_no_buttons_loop(tweets_95)

In [5]:
with open("/content/drive/MyDrive/tweet-archive/data/progress.json" , 'w') as file:
  json.dump(tweets_95, file)